In [1]:
import heapq

In [ ]:
with open("input-example.txt") as f:
    lines = f.readlines()
lines = [l.strip() for l in lines]
lines[:5]

In [ ]:
inc_bytes = []
for l in lines:
    b = l.split(",")
    inc_bytes.append((int(b[0]), int(b[1])))
inc_bytes[:3]

In [4]:
dirs = [(-1, 0), (0, 1), (1, 0), (0, -1)]

# Get dirs to turn to (180 makes no sense)
turn_dict = {
    (-1, 0): [(0, -1), (0, 1)],
    (0, 1): [(-1, 0), (1, 0)],
    (1, 0): [(0, 1), (0, -1)],
    (0, -1): [(1, 0), (-1, 0)],
}


def walk(pos, dir):
    return (pos[0] + dir[0], pos[1] + dir[1])


def in_bounds(pos, field):
    return (
        pos[0] >= 0 and pos[0] < len(field) and pos[1] >= 0 and pos[1] < len(field[0])
    )


def print_field(
    field,
):
    i = 0
    for l in field:
        j = 0
        for p in l:
            print(p, end="")
            j += 1
        print()
        i += 1

In [ ]:
width, height = (7, 7)
falling_bytes = 12

# uncomment for full input
# width, height = (71,71)
# falling_bytes = 1024

field = []
for i in range(0, height):
    l = []
    for j in range(0, width):
        l.append(".")
    field.append(l)


i = 0
for b in inc_bytes:
    if i >= falling_bytes:
        break
    # print(b)
    field[b[1]][b[0]] = "#"

    i += 1
print_field(field)

In [6]:
start_node = (0, 0)
target_node = (height - 1, width - 1)

visited_nodes = set()

In [7]:
def dijkstra(start, target, field):
    priority_queue = []
    heapq.heappush(priority_queue, (0, start))
    prev_nodes = {}
    distances = {}

    distances[start] = 0
    prev_nodes[start] = None
    visited_nodes = set()

    while priority_queue:
        current_distance, current_node = heapq.heappop(priority_queue)
        visited_nodes.add(current_node)
        if current_node == target:
            break

        if current_distance > distances.get(current_node, float("inf")):
            continue

        # Get neighbours
        neighbours = []
        # print(current_node)
        for dir in dirs:
            next_node = walk(current_node, dir)
            if (
                in_bounds(next_node, field)
                and field[next_node[0]][next_node[1]] != "#"
                and next_node not in visited_nodes
            ):
                neighbours.append((1, next_node))

        for weight, neighbor in neighbours:
            distance = current_distance + weight
            if distance < distances.get(neighbor, float("inf")):
                distances[neighbor] = distance
                prev_nodes[neighbor] = current_node
                heapq.heappush(priority_queue, (distance, neighbor))

    return distances, prev_nodes


distances, prev_nodes = dijkstra(start_node, target_node, field)
# print(distances, prev_nodes)

In [ ]:
distances[target_node]

In [9]:
# part 2

In [ ]:
# Do dijkstra after each fallen byte and check if there is a distance to the target
# If not, the path is blocked
# This is not optimal at all but still fast enough

width, height = (7, 7)
falling_bytes = 12

# uncomment for full input
# width, height = (71,71)
# falling_bytes = 1024

field = []
for i in range(0, height):
    l = []
    for j in range(0, width):
        l.append(".")
    field.append(l)

i = 0
for b in inc_bytes:
    if i >= falling_bytes:
        break
    # print(b)
    field[b[1]][b[0]] = "#"

    i += 1
# print_field(field)

for falling_bytes_ext in range(falling_bytes, len(inc_bytes) - 1):
    fb = inc_bytes[falling_bytes_ext]
    # print("Adding ", fb)
    
    field[fb[1]][fb[0]] = "#"
    distances, prev_nodes = dijkstra(start_node, target_node, field)

    if not distances.get(target_node):
        print(inc_bytes[falling_bytes_ext])
        break